In [1]:
from concurrent.futures import ThreadPoolExecutor
from bs4 import BeautifulSoup, SoupStrainer
import pandas as pd
import requests

In [7]:
def get_article_links(url):
    response = session.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, "html.parser", parse_only=SoupStrainer('a', class_="trunk2"))
    return (f"https://www.epochtimesviet.com{link['href']}" for link in soup.find_all('a', class_="trunk2"))

In [8]:
def parse_article(url):
    try:
        response = session.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")
        title = soup.find("h1", class_="name post-title entry-title")
        title = title.text.strip() if title is not None else " "
        paragraphs = soup.find_all("div", class_="entry")
        article_content = [
            x.text.strip()
            for content in paragraphs
            for x in content.find_all('span', {'style': 'font-weight: 400;'})
        ]
        return {
            "Link": url,
            "Title": title,
            "Content": " ".join(article_content),
            "label": "0",
        }
    except Exception as e:
        print(f"Error while parsing {url}: {e}")
        return None

In [9]:
data_Frame = []

with requests.Session() as session, ThreadPoolExecutor() as executor:
    urls = [f'https://www.epochtimesviet.com/my-trung.html/page/{x}' for x in range(1, 5)]
    article_links = (link for url in urls for link in get_article_links(url))
    parsed_articles = executor.map(parse_article, article_links)
    data_Frame = [article for article in parsed_articles if article is not None]

df = pd.DataFrame(data_Frame)
df.to_csv('xxx.csv', sep='\t', encoding='utf-8', index=False)